In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torchvision import transforms,datasets
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image


In [2]:
from create import create_dataloaders

In [3]:
img_list_path=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images.txt'

In [9]:
import os
import shutil

# Ruta de la carpeta que contiene los archivos .txt
source_dir = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images'

# Crear la carpeta 'labels' si no existe
labels_dir = os.path.join(source_dir, 'labels')
os.makedirs(labels_dir, exist_ok=True)

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(source_dir):
    if filename.endswith('.txt'):  # Si es un archivo .txt
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(labels_dir, filename)
        
        # Mover el archivo .txt a la carpeta 'labels'
        shutil.copy(source_path, destination_path)

print(f'Todos los archivos .txt se han movido a {labels_dir}')


Todos los archivos .txt se han movido a C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images\labels


In [4]:
import os

def create_images_txt(image_folder, output_txt):
    # Abrir el archivo para escribir
    with open(output_txt, "w") as file:
        # Recorrer todos los archivos en el folder de imágenes
        for filename in os.listdir(image_folder):
            # Si el archivo es una imagen (.jpg o .jpeg o .png)
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                # Escribir la ruta completa de la imagen en el archivo
                img_path = os.path.join(image_folder, filename)
                file.write(f"{img_path}\n")

# Especifica la carpeta de imágenes y el archivo de salida
image_folder = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images'
output_txt = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images.txt'

create_images_txt(image_folder, output_txt)


In [5]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset
from utils import encode


In [6]:
class YoloDataset(Dataset):
    def __init__(self, img_list_path, S, B, num_classes, transforms=None, img_box_transforms=None, eval_mode=False):
        with open(img_list_path, "r") as img_list_file:
            self.img_filenames = img_list_file.readlines()
            
        self.img_filenames = list(map(lambda x:x.strip(), self.img_filenames))
        self.label_files = []
        for path in self.img_filenames:
            image_dir = os.path.dirname(path)
            label_dir = "labels".join(image_dir.rsplit("images", 1))
            assert label_dir != image_dir, \
                f"Image path must contain a folder named 'images'! \n'{image_dir}'"
            label_file = os.path.join(r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images', os.path.basename(path))
            label_file = os.path.splitext(label_file)[0] + '.txt'
            self.label_files.append(label_file)

        self.transforms = transforms
        self.img_box_transforms = img_box_transforms
        
        self.S = S
        self.B = B
        self.num_classes = num_classes
        self.eval_mode = eval_mode

    def eval(self, eval_mode=True):
        self.eval_mode = eval_mode
        return

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        # read image
        img_filename = self.img_filenames[idx]
        img = Image.open(img_filename, mode='r')
        if self.transforms is not None:
            img = self.transforms(img)

        # read each image's corresponding label (.txt)
        labels = []
        with open(self.label_files[idx], 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line == '\n':
                    continue
                line = line.strip().split(' ')
                # convert xywh str to float, class str to int
                c, x, y, w, h = int(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])
                if self.eval_mode: 
                    labels.append((x, y, w, h, 1.0, c))
                else:
                    labels.append((x, y, w, h, c))
                
        if self.img_box_transforms is not None:
            for t in self.img_box_transforms:
                img, labels = t(img, labels)
        
        if self.eval_mode: 
            return img, torch.Tensor(labels)

        encoded_labels = encode(labels, self.S, self.B, self.num_classes)  # convert label list to encoded label
        encoded_labels = torch.Tensor(encoded_labels)
        return img, encoded_labels


In [10]:
dataset = YoloDataset(img_list_path, 7, 2, 2,)

In [11]:
# get data loader
train_loader, val_loader = create_dataloaders(img_list_path, 0.8, 0.2, 16, 448, 7, 2, 2)

In [13]:
next(iter(train_loader))[0]

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [3]:

# Abre tu imagen usando PIL
image = Image.open('image3.jpg')

In [4]:
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv, self).__init__()
        # Definimos la convolución
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        # Usamos ReLU como función de activación
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Aplicamos la convolución y luego la activación ReLU
        x = self.conv(x)
        x = self.relu(x)
        return x
# Definición de la clase Conv para una capa convolucional personalizada con LRN
class ConvLRN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvLRN, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.lrn = nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0)  # LRN

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.lrn(x)  # Aplicar LRN después de la activación
        return x
    
class ConvBN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0):
        super(ConvBN, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)  # Batch Normalization
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
        
class MaxPooling(nn.Module):
    def __init__(self,kernel_size=3,stride=2,padding=0):
        super(MaxPooling,self).__init__()
        self.maxpool=nn.MaxPool2d(kernel_size=kernel_size,stride=stride,padding=padding)
        
    def forward(self,x):
        x=self.maxpool(x)
        return x


In [7]:
class YOLOV1(nn.Module):
    def __init__(self,S,B,num_classes):
        super(YOLOV1,self).__init__()
        self.S = S
        self.B = B
        self.num_classes = num_classes
        
        self.layer1=Conv(kernel_size=7,in_channels=3,out_channels=64,stride=2)
        self.maxpool1=MaxPooling(kernel_size=2,stride=2)
        
        self.layer2=Conv(kernel_size=3,in_channels=64,out_channels=192,stride=1)
        self.maxpool2=MaxPooling(kernel_size=2,stride=2)
        
        self.layer3=Conv(kernel_size=1,in_channels=192,out_channels=128,stride=1)
        self.layer4=Conv(kernel_size=3,in_channels=128,out_channels=256,stride=1)
        self.layer5=Conv(kernel_size=1,in_channels=256,out_channels=256,stride=1)
        self.layer6=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        self.maxpool3=MaxPooling(kernel_size=2,stride=2)
        
        
        
        self.layer7=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer8=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer9=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer10=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer11=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer12=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer13=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer14=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer15=Conv(kernel_size=1,in_channels=512,out_channels=512,stride=1)
        self.layer16=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        
        
        self.maxpool4=MaxPooling(kernel_size=2,stride=2)
        
        
        
        self.layer17=Conv(kernel_size=1,in_channels=1024,out_channels=512,stride=1)
        self.layer18=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        self.layer19=Conv(kernel_size=1,in_channels=1024,out_channels=512,stride=1)
        self.layer20=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        self.layer21=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        
        self.layer22=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=2)
        
        self.layer23=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        self.layer24=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * 9* 9, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, self.S * self.S * (self.B * 5 + self.num_classes))  # Output de 7x7x30
        )
            
    def forward(self, x):
        x=self.layer1(x)
        x=self.maxpool1(x)
        
        x=self.layer2(x)
        x=self.maxpool2(x)
        
        
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=self.layer6(x)
        x=self.maxpool3(x)
        
        x=self.layer7(x)
        x=self.layer8(x)
        
        x=self.layer9(x)
        x=self.layer10(x)
        
        x=self.layer11(x)
        x=self.layer12(x)
        
        x=self.layer13(x)
        x=self.layer14(x)
        
        x=self.layer15(x)
        x=self.layer16(x)
        x=self.maxpool4(x)
        
        x=self.layer17(x)
        x=self.layer18(x)
        
        x=self.layer19(x)
        x=self.layer20(x)
        
        x=self.layer21(x)
        x=self.layer22(x)
        
        x=self.layer23(x)
        x=self.layer24(x)
        
        # Aplana el tensor para que tenga el tamaño adecuado

        
        x=self.fc1(x)
        
          # Reestructuramos la salida a 7x7x30
        x = x.reshape(-1, self.S, self.S, self.B * 5 + self.num_classes)
        
        return x
        
        
            
            
            


In [9]:
  
    
torch.manual_seed(42)
np.random.seed(42)

model = YOLOV1(7,2,2)

In [ ]:
import argparse
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim import SGD
from torch.optim.lr_scheduler import OneCycleLR
from torchvision import utils
from torch.utils.tensorboard import SummaryWriter

from utils import YoloLoss, load_yaml, metrics
from datasets import create_dataloaders

parser = argparse.ArgumentParser(description='YOLO')
parser.add_argument("--type", "-t", default="ms", help="model type", type=str)
parser.add_argument("--cfg", "-c", default="config/model.yaml", help="model config file", type=str)
parser.add_argument("--weights", "-w", default="", help="model weights", type=str)
parser.add_argument("--dataset", "-d", default="config/dataset.yaml", help="dataset config file", type=str)
parser.add_argument("--epochs", "-e", default=135, help="number of epochs", type=int)
parser.add_argument("--lr", "-lr", default=0.0005, help="learning rate", type=float)
parser.add_argument("--batch_size", "-bs", default=64, help="batch size", type=int)
parser.add_argument("--output", "-o", default="output", help="output dir", type=str)
parser.add_argument("--save", "-sv", default=5, help="save model checkpoint every #nth epoch", type=int)
parser.add_argument('--tboard', action='store_true', default=True, help='use tensorboard')
parser.add_argument("--cuda", "-cu", action='store_true', default=False, help='use cuda')
parser.add_argument("--sched_lr", "-sc", action='store_true', default=False, help='use scheduler')

# parse arguments
args = parser.parse_args()

torch.manual_seed(32)

def train(model, train_loader, optimizer, epoch, scheduler, device, S, B, train_loss_lst, writer):
    model.train()  # Set the module in training mode
    train_loss = 0
    pbar = tqdm(train_loader, leave=True)
    for batch_idx, (inputs, labels) in enumerate(pbar):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        # back prop
        criterion = YoloLoss(S, B)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if args.sched_lr:
            scheduler.step()
        train_loss += loss.item()

        # log batch0 images
        if batch_idx == 0 and epoch == 0:
            inputs = inputs.cpu()  # convert to cpu
            img_grid = utils.make_grid(inputs)
            writer.add_image('image batch0', img_grid, 0)

        # print loss and accuracy
        pbar.set_description(f"[Epoch {epoch+1}] loss = {train_loss/(batch_idx+1):.03f}")

    # record training loss
    train_loss /= len(pbar)
    train_loss_lst.append(train_loss)

    tqdm.write(f"Epoch {epoch} training summary -- loss = {train_loss:.03f}")
    return train_loss_lst


def validate(model, val_loader, device, S, B, valid_loss_list):
    model.eval()  # Sets the module in evaluation mode
    val_loss = 0
    pbar = tqdm(val_loader, leave=True)
    with torch.no_grad(): # without gradient calculation
        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            output = model(data)
            criterion = YoloLoss(S, B)
            val_loss += criterion(output, target).item()

    val_loss /= len(val_loader)

    tqdm.write(f"Epoch {epoch} validation summary -- loss = {val_loss:.03f}")
   
    valid_loss_list.append(val_loss)
    return valid_loss_list


cfg = load_yaml(args.cfg)
print('cfg:', cfg)
dataset = load_yaml(args.dataset)
print('dataset:', dataset)

img_list_path = dataset['images']
S, B, num_classes, input_size = cfg['S'], cfg['B'], cfg['num_classes'], cfg['input_size']

# create output file folder
start = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(time.time()))
output_dir = os.path.join(args.output, 'train', start)
os.makedirs(output_dir)

if args.cuda:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")

# build model
model = YOLOV1(S, B, num_classes).to(device)

# get data loader
train_loader, val_loader = create_dataloaders(img_list_path, 0.8, 0.2, args.batch_size, input_size, S, B, num_classes)

optimizer = SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.0005)

scheduler = None
if args.sched_lr:
    scheduler = OneCycleLR(optimizer, max_lr=0.01, epochs=args.epochs, steps_per_epoch=len(train_loader), anneal_strategy='cos')

train_loss_lst, valid_loss_list = [], []

print('using tensorboard...')
print('to run tesnsorboard server, run: \'tensorboard --logdir output/train/ --bind_all\'')
os.makedirs(output_dir, exist_ok=True)
writer = SummaryWriter(output_dir)

# train epoch
for epoch in range(args.epochs):
    train_loss_lst = train(model, train_loader, optimizer, epoch, scheduler, device, S, B, train_loss_lst, writer)
    valid_loss_list = validate(model, val_loader, device, S, B, valid_loss_list)
    writer.add_scalar('Loss/train', np.average(train_loss_lst), epoch)
    writer.add_scalar('Loss/validate', np.average(valid_loss_list), epoch)

    # save model weight every save_freq epoch
    if epoch % args.save_freq == 0:
        torch.save(model.state_dict(), os.path.join(output_dir, 'epoch' + str(epoch) + '.pt'))


In [ ]:
# save model
torch.save(model.state_dict(), os.path.join(output_dir, 'last.pt'))

# save loss plot
fig = plt.figure()
plt.plot(range(args.epochs), train_loss_lst, 'g', label='train loss')
plt.plot(range(args.epochs), valid_loss_list, 'r', label='val loss')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.savefig(os.path.join(output_dir, 'loss_plot.jpg'))
plt.close(fig)


In [10]:

# Define la transformación para redimensionar la imagen a 227x227
resize_transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Cambiamos el tamaño a 227x227
    transforms.ToTensor()           # Convertimos la imagen a un tensor
])

# Aplica la transformación a la imagen
input_image = resize_transform(image)

# Añade una dimensión para el batch (porque AlexNet espera el tamaño (batch_size, 3, 227, 227))
input_image = input_image.unsqueeze(0)  # Esto convierte la imagen a (1, 3, 227, 227)

# Pasa la imagen a través del modelo AlexNet
output = model(input_image)

# Imprime la forma de la salida
print(output.shape)



torch.Size([1, 7, 7, 12])


In [36]:
#loss
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class Loss(nn.Module):

    def __init__(self, feature_size=7, num_bboxes=2, num_classes=2, lambda_coord=5.0, lambda_noobj=0.5):
        """ Constructor.
        Args:
            feature_size: (int) size of input feature map.
            num_bboxes: (int) number of bboxes per each cell.
            num_classes: (int) number of the object classes.
            lambda_coord: (float) weight for bbox location/size losses.
            lambda_noobj: (float) weight for no-objectness loss.
        """
        super(Loss, self).__init__()

        self.S = feature_size
        self.B = num_bboxes
        self.C = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj


    def compute_iou(self, bbox1, bbox2):
        """ Compute the IoU (Intersection over Union) of two set of bboxes, each bbox format: [x1, y1, x2, y2].
        Args:
            bbox1: (Tensor) bounding bboxes, sized [N, 4].
            bbox2: (Tensor) bounding bboxes, sized [M, 4].
        Returns:
            (Tensor) IoU, sized [N, M].
        """
        N = bbox1.size(0)
        M = bbox2.size(0)

        # Compute left-top coordinate of the intersections
        lt = torch.max(
            bbox1[:, :2].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
            bbox2[:, :2].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
        )
        # Conpute right-bottom coordinate of the intersections
        rb = torch.min(
            bbox1[:, 2:].unsqueeze(1).expand(N, M, 2), # [N, 2] -> [N, 1, 2] -> [N, M, 2]
            bbox2[:, 2:].unsqueeze(0).expand(N, M, 2)  # [M, 2] -> [1, M, 2] -> [N, M, 2]
        )
        # Compute area of the intersections from the coordinates
        wh = rb - lt   # width and height of the intersection, [N, M, 2]
        wh[wh < 0] = 0 # clip at 0
        inter = wh[:, :, 0] * wh[:, :, 1] # [N, M]

        # Compute area of the bboxes
        area1 = (bbox1[:, 2] - bbox1[:, 0]) * (bbox1[:, 3] - bbox1[:, 1]) # [N, ]
        area2 = (bbox2[:, 2] - bbox2[:, 0]) * (bbox2[:, 3] - bbox2[:, 1]) # [M, ]
        area1 = area1.unsqueeze(1).expand_as(inter) # [N, ] -> [N, 1] -> [N, M]
        area2 = area2.unsqueeze(0).expand_as(inter) # [M, ] -> [1, M] -> [N, M]

        # Compute IoU from the areas
        union = area1 + area2 - inter # [N, M, 2]
        iou = inter / union           # [N, M, 2]

        return iou

    def forward(self, pred_tensor, target_tensor):
        """ Compute loss for YOLO training.
        Args:
            pred_tensor: (Tensor) predictions, sized [n_batch, S, S, Bx5+C], 5=len([x, y, w, h, conf]).
            target_tensor: (Tensor) targets, sized [n_batch, S, S, Bx5+C].
        Returns:
            (Tensor): loss, sized [1, ].
        """
        # TODO: Romove redundant dimensions for some Tensors.

        S, B, C = self.S, self.B, self.C
        N = 5 * B + C    # 5=len([x, y, w, h, conf]

        batch_size = pred_tensor.size(0)
        coord_mask = target_tensor[:, :, :, 4] > 0  # mask for the cells which contain objects. [n_batch, S, S]
        noobj_mask = target_tensor[:, :, :, 4] == 0 # mask for the cells which do not contain objects. [n_batch, S, S]
        coord_mask = coord_mask.unsqueeze(-1).expand_as(target_tensor) # [n_batch, S, S] -> [n_batch, S, S, N]
        noobj_mask = noobj_mask.unsqueeze(-1).expand_as(target_tensor) # [n_batch, S, S] -> [n_batch, S, S, N]

        coord_pred = pred_tensor[coord_mask].view(-1, N)            # pred tensor on the cells which contain objects. [n_coord, N]
                                                                    # n_coord: number of the cells which contain objects.
        bbox_pred = coord_pred[:, :5*B].contiguous().view(-1, 5)    # [n_coord x B, 5=len([x, y, w, h, conf])]
        class_pred = coord_pred[:, 5*B:]                            # [n_coord, C]

        coord_target = target_tensor[coord_mask].view(-1, N)        # target tensor on the cells which contain objects. [n_coord, N]
                                                                    # n_coord: number of the cells which contain objects.
        bbox_target = coord_target[:, :5*B].contiguous().view(-1, 5)# [n_coord x B, 5=len([x, y, w, h, conf])]
        class_target = coord_target[:, 5*B:]                        # [n_coord, C]

        # Compute loss for the cells with no object bbox.
        noobj_pred = pred_tensor[noobj_mask].view(-1, N)        # pred tensor on the cells which do not contain objects. [n_noobj, N]
                                                                # n_noobj: number of the cells which do not contain objects.
        noobj_target = target_tensor[noobj_mask].view(-1, N)    # target tensor on the cells which do not contain objects. [n_noobj, N]
                                                                # n_noobj: number of the cells which do not contain objects.
        noobj_conf_mask = torch.cuda.ByteTensor(noobj_pred.size()).fill_(0) # [n_noobj, N]
        for b in range(B):
            noobj_conf_mask[:, 4 + b*5] = 1 # noobj_conf_mask[:, 4] = 1; noobj_conf_mask[:, 9] = 1
        noobj_pred_conf = noobj_pred[noobj_conf_mask]       # [n_noobj, 2=len([conf1, conf2])]
        noobj_target_conf = noobj_target[noobj_conf_mask]   # [n_noobj, 2=len([conf1, conf2])]
        loss_noobj = F.mse_loss(noobj_pred_conf, noobj_target_conf, reduction='sum')

        # Compute loss for the cells with objects.
        coord_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(0)    # [n_coord x B, 5]
        coord_not_response_mask = torch.cuda.ByteTensor(bbox_target.size()).fill_(1)# [n_coord x B, 5]
        bbox_target_iou = torch.zeros(bbox_target.size()).cuda()                    # [n_coord x B, 5], only the last 1=(conf,) is used

        # Choose the predicted bbox having the highest IoU for each target bbox.
        for i in range(0, bbox_target.size(0), B):
            pred = bbox_pred[i:i+B] # predicted bboxes at i-th cell, [B, 5=len([x, y, w, h, conf])]
            pred_xyxy = Variable(torch.FloatTensor(pred.size())) # [B, 5=len([x1, y1, x2, y2, conf])]
            # Because (center_x,center_y)=pred[:, 2] and (w,h)=pred[:,2:4] are normalized for cell-size and image-size respectively,
            # rescale (center_x,center_y) for the image-size to compute IoU correctly.
            pred_xyxy[:,  :2] = pred[:, :2]/float(S) - 0.5 * pred[:, 2:4]
            pred_xyxy[:, 2:4] = pred[:, :2]/float(S) + 0.5 * pred[:, 2:4]

            target = bbox_target[i] # target bbox at i-th cell. Because target boxes contained by each cell are identical in current implementation, enough to extract the first one.
            target = bbox_target[i].view(-1, 5) # target bbox at i-th cell, [1, 5=len([x, y, w, h, conf])]
            target_xyxy = Variable(torch.FloatTensor(target.size())) # [1, 5=len([x1, y1, x2, y2, conf])]
            # Because (center_x,center_y)=target[:, 2] and (w,h)=target[:,2:4] are normalized for cell-size and image-size respectively,
            # rescale (center_x,center_y) for the image-size to compute IoU correctly.
            target_xyxy[:,  :2] = target[:, :2]/float(S) - 0.5 * target[:, 2:4]
            target_xyxy[:, 2:4] = target[:, :2]/float(S) + 0.5 * target[:, 2:4]

            iou = self.compute_iou(pred_xyxy[:, :4], target_xyxy[:, :4]) # [B, 1]
            max_iou, max_index = iou.max(0)
            max_index = max_index.data.cuda()

            coord_response_mask[i+max_index] = 1
            coord_not_response_mask[i+max_index] = 0

            # "we want the confidence score to equal the intersection over union (IOU) between the predicted box and the ground truth"
            # from the original paper of YOLO.
            bbox_target_iou[i+max_index, torch.LongTensor([4]).cuda()] = (max_iou).data.cuda()
        bbox_target_iou = Variable(bbox_target_iou).cuda()

        # BBox location/size and objectness loss for the response bboxes.
        bbox_pred_response = bbox_pred[coord_response_mask].view(-1, 5)      # [n_response, 5]
        bbox_target_response = bbox_target[coord_response_mask].view(-1, 5)  # [n_response, 5], only the first 4=(x, y, w, h) are used
        target_iou = bbox_target_iou[coord_response_mask].view(-1, 5)        # [n_response, 5], only the last 1=(conf,) is used
        loss_xy = F.mse_loss(bbox_pred_response[:, :2], bbox_target_response[:, :2], reduction='sum')
        loss_wh = F.mse_loss(torch.sqrt(bbox_pred_response[:, 2:4]), torch.sqrt(bbox_target_response[:, 2:4]), reduction='sum')
        loss_obj = F.mse_loss(bbox_pred_response[:, 4], target_iou[:, 4], reduction='sum')

        # Class probability loss for the cells which contain objects.
        loss_class = F.mse_loss(class_pred, class_target, reduction='sum')

        # Total loss
        loss = self.lambda_coord * (loss_xy + loss_wh) + loss_obj + self.lambda_noobj * loss_noobj + loss_class
        loss = loss / float(batch_size)

        return loss

In [48]:
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np
import torch
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, image_dir, split='train', split_ratio=0.8, transform=None):
        self.image_dir = image_dir
        self.split = split
        self.split_ratio = split_ratio
        self.transform = transform
        
        # List all files in the directory
        all_files = os.listdir(image_dir)
        all_images = [file_name for file_name in all_files if file_name.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Split into train and validation
        num_images = len(all_images)
        split_index = int(num_images * split_ratio)
        
        if split == 'train':
            self.image_paths = all_images[:split_index]
        else:
            self.image_paths = all_images[split_index:]
        
        self.image_paths = [os.path.join(image_dir, file_name) for file_name in self.image_paths]
        self.label_paths = [os.path.join(image_dir, file_name.rsplit('.', 1)[0] + '.txt') for file_name in self.image_paths]
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label_path = self.label_paths[index]
        
        # Load image
        image = Image.open(image_path).convert('RGB')
        image = np.array(image).astype(np.float32) / 255.0
        
        # Load label
        boxes = []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:])

                # Convert normalized coordinates to absolute
                boxes.append([class_id, x_center, y_center, width, height])
        
        if len(boxes) > 0:
            boxes = np.array(boxes, dtype=np.float32)
        else:
            boxes = np.zeros((0, 5), dtype=np.float32)
        
        # Apply transformation if provided
        if self.transform:
            image = self.transform(image)
        
        return image, boxes



In [49]:
import torch

def collate_fn(batch):
    imgs, targets = zip(*batch)
    
    # Convertir imágenes a tensor si son numpy.ndarray
    if isinstance(imgs[0], np.ndarray):
        imgs = [torch.tensor(img, dtype=torch.float32).permute(2, 0, 1) for img in imgs]
    
    # Convertir objetivos a tensor si son numpy.ndarray
    if isinstance(targets[0], np.ndarray):
        targets = [torch.tensor(target, dtype=torch.float32) for target in targets]
    
    imgs = torch.stack(imgs, 0)  # Apilar imágenes en un tensor
    targets = torch.stack(targets, 0)  # Apilar objetivos en un tensor
    
    return imgs, targets




In [50]:
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import numpy as np
import math
from datetime import datetime

# Check if GPU devices are available.
use_gpu = torch.cuda.is_available()
print('CUDA available:', use_gpu)
if use_gpu:
    
    print('CUDA current_device: {}'.format(torch.cuda.current_device()))
    print('CUDA device_count: {}'.format(torch.cuda.device_count()))

# Path to data dir.
image_dir = 'C:/Users/ASUS RYZEN 7/Documents/PROYECTOS/Computer vision/animas/agri_data/data'

# Path to checkpoint file containing pre-trained DarkNet weight.
checkpoint_path = 'weights/darknet/model_best.pth.tar'

# Frequency to print/log the results.
print_freq = 5
tb_log_freq = 5

# Training hyper parameters.
init_lr = 0.001
base_lr = 0.01
momentum = 0.9
weight_decay = 5.0e-4
num_epochs = 4
batch_size = 16

# Learning rate scheduling.
def update_lr(optimizer, epoch, burnin_base, burnin_exp=4.0):
    if epoch == 0:
        lr = init_lr + (base_lr - init_lr) * math.pow(burnin_base, burnin_exp)
    elif epoch == 1:
        lr = base_lr
    elif epoch == 75:
        lr = 0.001
    elif epoch == 105:
        lr = 0.0001
    else:
        return

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# Load YOLO model.
yolo = model

# Setup loss and optimizer.
# Crear el modelo


# Calcular el feature size pasando un tamaño de entrada (por ejemplo, 448x448)
input_size = 448  # Asumiendo que esta es la resolución de entrada
feature_size = yolo.get_feature_size(input_size)

# Configurar el criterio de pérdida
criterion = Loss(feature_size=feature_size)

optimizer = torch.optim.SGD(yolo.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay)




# Definir la transformación para el dataset
transform = transforms.Compose([
    transforms.Resize((448, 448)),  # Redimensionar imagen
    transforms.ToTensor(),  # Convertir imagen a tensor
])

train_dataset = CustomDataset(image_dir, split='train', transform=transform)
val_dataset = CustomDataset(image_dir, split='val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print('Number of training images: ', len(train_dataset))
print('Number of validation images: ', len(val_dataset))


CUDA available: False
Number of training images:  1040
Number of validation images:  260


In [51]:
next(iter(train_loader))

TypeError: Unexpected type <class 'numpy.ndarray'>

In [42]:
from tqdm import tqdm
import torch

for epoch in range(num_epochs):
    print('\n')
    print(f'Starting epoch {epoch + 1} / {num_epochs}')

    # Training.
    yolo.train()
    total_loss = 0.0
    total_batch = 0

    # Añadir barra de progreso para entrenamiento
    train_loader_tqdm = tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', ncols=100)

    for i, (imgs, targets) in enumerate(train_loader_tqdm):
        # Move to GPU if available
        if use_gpu:
            imgs = imgs.to(device)
            targets = targets.to(device)

        # Update learning rate.
        update_lr(optimizer, epoch, float(i) / float(len(train_loader) - 1))
        lr = get_lr(optimizer)

        # Forward pass to compute loss.
        preds = yolo(imgs)
        loss = criterion(preds, targets)
        loss_this_iter = loss.item()
        total_loss += loss_this_iter * imgs.size(0)
        total_batch += imgs.size(0)

        # Backward pass to update model weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the progress bar.
        train_loader_tqdm.set_postfix({
            'Loss': loss_this_iter, 
            'Avg Loss': total_loss / float(total_batch), 
            'LR': lr
        })

    # Validation.
    yolo.eval()
    val_loss = 0.0
    total_batch = 0

    # Añadir barra de progreso para validación
    val_loader_tqdm = tqdm(val_loader, desc=f'Validating Epoch {epoch+1}/{num_epochs}', ncols=100)

    for i, (imgs, targets) in enumerate(val_loader_tqdm):
        # Move to GPU if available
        if use_gpu:
            imgs = imgs.to(device)
            targets = targets.to(device)

        # Forward pass to compute validation loss.
        with torch.no_grad():
            preds = yolo(imgs)
            loss = criterion(preds, targets)
            loss_this_iter = loss.item()
            val_loss += loss_this_iter * imgs.size(0)
            total_batch += imgs.size(0)

    val_loss /= float(total_batch)

    # Print validation loss.
    print(f'Epoch [{epoch + 1}/{num_epochs}], Val Loss: {val_loss:.4f}')





Starting epoch 1 / 4


Training Epoch 1/4:   0%|                                                    | 0/65 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[16, 512, 512, 3] to have 3 channels, but got 512 channels instead

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable

import os
import numpy as np
import math
from datetime import datetime


# Check if GPU devices are available.
use_gpu = torch.cuda.is_available()
assert use_gpu, 'Current implementation does not support CPU mode. Enable CUDA.'
print('CUDA current_device: {}'.format(torch.cuda.current_device()))
print('CUDA device_count: {}'.format(torch.cuda.device_count()))

# Path to data dir.
image_dir = 'data/VOC_allimgs/'

# Path to label files.
train_label = ('data/voc2007.txt', 'data/voc2012.txt')
val_label = 'data/voc2007test.txt'

# Path to checkpoint file containing pre-trained DarkNet weight.
checkpoint_path = 'weights/darknet/model_best.pth.tar'

# Frequency to print/log the results.
print_freq = 5
tb_log_freq = 5

# Training hyper parameters.
init_lr = 0.001
base_lr = 0.01
momentum = 0.9
weight_decay = 5.0e-4
num_epochs = 135
batch_size = 64

# Learning rate scheduling.
def update_lr(optimizer, epoch, burnin_base, burnin_exp=4.0):
    if epoch == 0:
        lr = init_lr + (base_lr - init_lr) * math.pow(burnin_base, burnin_exp)
    elif epoch == 1:
        lr = base_lr
    elif epoch == 75:
        lr = 0.001
    elif epoch == 105:
        lr = 0.0001
    else:
        return

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


# Load YOLO model.
yolo = model

# Setup loss and optimizer.
criterion = Loss(feature_size=yolo.feature_size)
optimizer = torch.optim.SGD(yolo.parameters(), lr=init_lr, momentum=momentum, weight_decay=weight_decay)

# Load Pascal-VOC dataset.
train_dataset = VOCDataset(True, image_dir, train_label)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

val_dataset = VOCDataset(False, image_dir, val_label)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print('Number of training images: ', len(train_dataset))



for epoch in range(num_epochs):
    print('\n')
    print('Starting epoch {} / {}'.format(epoch, num_epochs))

    # Training.
    yolo.train()
    total_loss = 0.0
    total_batch = 0

    for i, (imgs, targets) in enumerate(train_loader):
        # Update learning rate.
        update_lr(optimizer, epoch, float(i) / float(len(train_loader) - 1))
        lr = get_lr(optimizer)

        # Load data as a batch.
        batch_size_this_iter = imgs.size(0)
        imgs = Variable(imgs)
        targets = Variable(targets)
        imgs, targets = imgs.cuda(), targets.cuda()

        # Forward to compute loss.
        preds = yolo(imgs)
        loss = criterion(preds, targets)
        loss_this_iter = loss.item()
        total_loss += loss_this_iter * batch_size_this_iter
        total_batch += batch_size_this_iter

        # Backward to update model weight.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print current loss.
        if i % print_freq == 0:
            print('Epoch [%d/%d], Iter [%d/%d], LR: %.6f, Loss: %.4f, Average Loss: %.4f'
            % (epoch, num_epochs, i, len(train_loader), lr, loss_this_iter, total_loss / float(total_batch)))

        
    # Validation.
    yolo.eval()
    val_loss = 0.0
    total_batch = 0

    for i, (imgs, targets) in enumerate(val_loader):
        # Load data as a batch.
        batch_size_this_iter = imgs.size(0)
        imgs = Variable(imgs)
        targets = Variable(targets)
        imgs, targets = imgs.cuda(), targets.cuda()

        # Forward to compute validation loss.
        with torch.no_grad():
            preds = yolo(imgs)
        loss = criterion(preds, targets)
        loss_this_iter = loss.item()
        val_loss += loss_this_iter * batch_size_this_iter
        total_batch += batch_size_this_iter
    val_loss /= float(total_batch)

    
    # Print.
    print('Epoch [%d/%d], Val Loss: %.4f, Best Val Loss: %.4f'
    % (epoch + 1, num_epochs, val_loss))

    